# Advent of Code 2025

### Common code

In [1]:
import re

from typing import Literal, Iterable

def read_input_lines(day: int) -> Iterable[str]:
    """
    Reads an input file for a given day and difficulty, yielding one line at a time.
    """
    with open(f"input/{day:02}", 'r') as file:
        yield from file.readlines()

def parse_input(day: int, pattern: re.Pattern) -> Iterable[re.Match]:
    """
    Parse an input file line-by-line, yielding the given regex pattern matches.
    """
    for line in read_input_lines(day):
        yield pattern.match(line)

def natural_mod(x: int, m: int) -> int:
    """
    "Natural numbers" modulo, as such that the modulo of negative numbers will instead loop back to positive.
    """
    return (x % m + m) % m


## Day 1 - Secret entrance

In [2]:
input_pattern = re.compile(r"(?P<direction>L|R)(?P<value>\d+)")

DIAL_START_POS: int = 50
DIAL_SIZE: int = 100

In [3]:
def easy_challenge() -> int:

    current_pos = DIAL_START_POS
    zero_passes = 0

    for line in parse_input(1, input_pattern):
        (direction, value) = line.groupdict().values()
        value = int(value)
        
        match direction:
            case 'L': current_pos = natural_mod(current_pos - value, DIAL_SIZE)
            case 'R': current_pos = natural_mod(current_pos + value, DIAL_SIZE)
            case _:
                return ValueError(f"'{direction}' is not a valid direction")
                
        if current_pos == 0:
            zero_passes += 1

    return zero_passes

easy_challenge()

1034

In [4]:
def hard_challenge() -> int:

    class Dial:

        current_pos: int
        zero_passes: int

        def __init__(self):
            self.current_pos = DIAL_START_POS
            self.zero_passes = 0

        def move(self, value: int, clockwise: bool) -> None:
            click = +1 if clockwise else -1
            for _ in range(value):
                self.current_pos += click

                if self.current_pos == DIAL_SIZE:
                    self.current_pos = 0
                    
                elif self.current_pos == -1:
                    self.current_pos = DIAL_SIZE - 1
                
                if self.current_pos == 0:
                    self.zero_passes += 1

    dial = Dial()

    for line in parse_input(1, input_pattern):
        (direction, value) = line.groupdict().values()
        value = int(value)

        match direction:
            case 'L': dial.move(value, False)
            case 'R': dial.move(value, True)
            case _:
                return ValueError(f"'{direction}' is not a valid direction")

    return dial.zero_passes

hard_challenge()

6166

## Day 2 - Gift Shop

In [5]:
from math import ceil

def parse_ranges() -> list[range]:
    line: str = next(read_input_lines(2))
    for range_ in line.split(","):
        (start, stop) = range_.split("-")
        yield range(int(start), int(stop) + 1)

def get_nlength_number(length: int) -> int:
    """
    Returns the smallest number of length `length`, as a str.
    """
    assert length > 0, "Length must be at least 1"
    return 10 ** (length - 1)

type NormalRange = range # Simple label to differentiate "normalized" ranges
def normalize(ranges: list[range]) -> list[NormalRange]:
    """
    Yield from ranges, splitting a range if its start and stop differ in number of digits,
    such that all ranges in the resulting collection have a start and stop of the same 10th power order.
    """
    for range_ in ranges:
        start_len = len(str(range_.start))
        stop_len = len(str(range_.stop - 1))

        start = range_.start
        sep = start
        for i in range(start_len, stop_len):
            sep = get_nlength_number(i + 1)
            yield range(start, sep)
            start = sep

        yield range(sep, range_.stop)

In [6]:
def easy_challenge() -> int:

    total: int = 0

    for range_ in normalize(parse_ranges()):
        number_length = len(str(range_.start))

        if number_length % 2 != 0:
            continue
        pattern_length = number_length // 2

        start_half = int(str(range_.start)[:pattern_length])
        stop_half = int(str(range_.stop - 1)[:pattern_length])
        for pattern in range(start_half, stop_half + 1):
            if (result := int(str(pattern) * 2)) in range_:
                total += result

    return total

easy_challenge()

23560874270

In [7]:
def hard_challenge() -> int:

    def find_patterned_numbers(range_: NormalRange) -> set[int]:
        """
        For a given "normal range", find all "patterned" numbers.
        """
        found = set[int]()
        
        number_length = len(str(range_.start))
        for pattern_length in range(1, number_length // 2 + 1):
            if number_length % pattern_length == 0:
                repeat = number_length // pattern_length
                found.update(
                    result
                    for pattern in range(
                        int(str(range_.start)[:pattern_length]),   # start
                        int(str(range_.stop - 1)[:pattern_length]) + 1 # stop
                    )
                    if (result := int(str(pattern) * repeat)) in range_
                )
                
        return found
    
    return sum(
        sum(find_patterned_numbers(range_))
        for range_ in normalize(parse_ranges())
    )

hard_challenge()

44143124633

## Day 3 - Lobby

In [8]:
type Bank = list[int]
line_pattern = re.compile(r"(\d+)")

def parse_banks() -> list[Bank]:
    for match in parse_input(3, line_pattern):
        yield [int(digit) for digit in match.group(1)]

In [9]:
def easy_challenge() -> int:

    total_joltage: int = 0

    for bank in parse_banks():

        first_digit_i = bank.index(max(bank[:-1]))

        second_bank = bank[first_digit_i + 1:]
        second_digit_i = first_digit_i + 1 + second_bank.index(max(second_bank))

        total_joltage += 10 * bank[first_digit_i] + bank[second_digit_i]

    return total_joltage

easy_challenge()

17229

In [10]:
BATTERIES_COUNT: int = 12

def hard_challenge() -> int:

    total_joltage: int = 0

    for bank in parse_banks():

        selected_batteries = list(range(
            len(bank) - BATTERIES_COUNT,
            len(bank)
        ))

        lower_bound = -1
        for i in range(BATTERIES_COUNT):
            for j in range(selected_batteries[i] - 1, lower_bound, -1):
                
                if bank[j] >= bank[selected_batteries[i]]:
                    selected_batteries[i] = j
                    
            lower_bound = selected_batteries[i]

        value = "".join(str(bank[i]) for i in selected_batteries)
        total_joltage += int(value)

    return total_joltage

hard_challenge()

170520923035051